In [67]:
from collections import deque
import numpy as np
import pandas as pd
from data_wrangling import df_possible_flights
from pathlib import Path


In [60]:
# distance
def distance_calc(lat1,lon1,lat2,lon2):
    earthRadius = 6371 #km

    dlon = float(lon2) - float(lon1)
    dlat = float(lat2) - float(lat1)
    a = np.sin(dlat/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    a = np.array([a])
    c = 2 * np.arctan(np.sqrt(a),np.sqrt(1-a))
    d = np.round(earthRadius * c,2)
    return(d)


In [61]:
def create_matrix(df, airports):
    num_airports = len(airports)

    # Create an adjacency matrix with zeros
    adjacency_matrix = np.zeros((num_airports, num_airports), dtype=float)

    # Fill the adjacency matrix based on the flights
    for index, row in df.iterrows():
        source_index = airports.index(row["Source Airport"])
        dest_index = airports.index(row["Destination Airport"])
        lat1, lon1 = row["Src Latitude"], row["Src Longitude"]
        lat2, lon2 = row["Dest Latitude"], row["Dest Longitude"]
        
        distance = distance_calc(lat1, lon1, lat2, lon2)
        adjacency_matrix[source_index][dest_index] = distance

    source = [airports.index('JFK'), airports.index('LGA')]
    sink = [airports.index('SFO')]

    return adjacency_matrix, source, sink


In [71]:
def ford_fulkerson(graph, sources, terminals):
    def residual_capacity(u, v):
        return graph[u][v] - flow[u][v]

    def augment(path):
        # Initialize min_capacity to an infinitely large number
        # this is similar to the relaxing schema seen in the Bellman-Ford Algorithm
        min_capacity = float('inf')

        # go through path array
        for i in range(len(path) - 1):
            # find the minimum capacity between the airports
            min_capacity = min(min_capacity, residual_capacity(path[i], path[i + 1]))

        # update flow of the edges and reverse edges along the path
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            flow[u][v] += min_capacity # update augmenting path
            flow[v][u] -= min_capacity # update flow on residual edges
        return min_capacity

    def bfs():
        visited = [False] * len(graph) # similar to the color system, but using True/False instead
        queue = deque(sources) # getting the first source in the list
        for source in sources:
            visited[source] = True

        # setting the sources as parents initially
        parent = {source: None for source in sources}

        # while there is a source in the queue, find flights
        # that connect from source to terminal
        while queue:
            # getting the first element in the queue
            u = queue.popleft()
            # go through the airports that are connected to the parent
            for v in range(len(graph)):
                # if it hasn't been visited and it can  be augmented
                if not visited[v] and residual_capacity(u, v) > 0:
                    # add it to the queue
                    queue.append(v)
                    visited[v] = True
                    # set the u the parent of v
                    parent[v] = u

                    # if v is a terminal then we will create a path for the
                    # flight
                    if v in terminals:
                        path = []
                        # adding cities into the path
                        while v is not None:
                            path.insert(0, v)
                            v = parent[v]
                        return path

        return None

    # Initialize flow to zero
    flow = np.zeros_like(graph)

    # Ford-Fulkerson algorithm
    total_flow = 0
    while True:
        augmenting_path = bfs()
        if augmenting_path is None:
            break
        min_capacity = augment(augmenting_path)
        total_flow += min_capacity
        print(f"Flight Route: {[airports[node] for node in augmenting_path]}, "
              f"total distance: {min_capacity} km")

    # Print the visited airports in the final path
    final_path = bfs()
    if final_path is not None:
        print("Final Path:", " --> ".join(airports[node] for node in final_path))

    print("Max Flow:", total_flow)


In [72]:
df = df_possible_flights
airports = sorted(set(df["Source Airport"]).union(set(df["Destination Airport"])))
print(airports)
adjacency_matrix, sources, sinks = create_matrix(df_possible_flights, airports)

adjacency_df = pd.DataFrame(adjacency_matrix, columns=airports, index=airports)

ford_fulkerson(adjacency_matrix, sources, sinks)

# csv_file_path = "/Users/yuhanburgess/Documents/GitHub/AGP2/csv_files/adjacency_matrix.csv"
# adjacency_df.to_csv(csv_file_path, index=True, header=True) 
# print("CSV file saved successfully.")


['ABQ', 'ACV', 'AKL', 'AMM', 'AMS', 'ANU', 'ARN', 'ATL', 'AUA', 'AUH', 'AUS', 'AZS', 'BCN', 'BDA', 'BFL', 'BGI', 'BGO', 'BGR', 'BHM', 'BJX', 'BNA', 'BOG', 'BOI', 'BOS', 'BQN', 'BRU', 'BTV', 'BUF', 'BUR', 'BWI', 'CAI', 'CAK', 'CCS', 'CDG', 'CEC', 'CHO', 'CHS', 'CIC', 'CLE', 'CLT', 'CMH', 'CMN', 'CPH', 'CTG', 'CUN', 'CVG', 'DAY', 'DCA', 'DEL', 'DEN', 'DFW', 'DKR', 'DOH', 'DSM', 'DTW', 'DUB', 'DUS', 'DXB', 'EUG', 'EWR', 'EZE', 'FAT', 'FCO', 'FLL', 'FRA', 'GCM', 'GDL', 'GEO', 'GIG', 'GND', 'GRR', 'GRU', 'GSO', 'GUA', 'GVA', 'GYE', 'HEL', 'HKG', 'HND', 'HNL', 'HOU', 'IAD', 'IAH', 'ICN', 'ILM', 'IND', 'IST', 'JAX', 'JED', 'JFK', 'JNB', 'KBP', 'KIN', 'KIX', 'KOA', 'KWI', 'LAS', 'LAX', 'LCY', 'LGA', 'LGB', 'LHE', 'LHR', 'LIH', 'LIM', 'LIR', 'LIT', 'LMT', 'LOS', 'LRM', 'MAD', 'MAN', 'MBJ', 'MCI', 'MCO', 'MDE', 'MEM', 'MEX', 'MFR', 'MIA', 'MKE', 'MLM', 'MNL', 'MOD', 'MRY', 'MSN', 'MSP', 'MSY', 'MUC', 'MXP', 'MYR', 'NAS', 'NCE', 'NRT', 'OAK', 'OGG', 'OKC', 'OMA', 'ORD', 'ORF', 'ORY', 'OSL', 'PAP'